## FastText 模型

### FastText模型适用的场景

FastText模型背后最主要的原则是单词的形态结构蕴含了单词背身的意义，传统的词表征方法比如Word2Vec往往没有考虑到该结构，对于富含结构信息的语言（German, Turkish），一个单词往往有很多种形式，每种形式可能极少出现，对于词表征的训练非常困难。

FastText通过将每个单词视为某个subword的集合来解决这种问题。subword通常通过单词的某几个字符来定义，单词的向量即为字符向量的和。

对比Word2Vec，FastText更适用于语料库较小的场景，Word2Vec在语义任务上表现得更好，随着语料库的增加，两者之间的差距逐渐减小。不过，FastText能处理OOV问题。

### 训练模型

同样使用Lee语料库来训练FastText模型：

In [1]:
import pprint
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

# corpus file location
corpus_file = datapath('lee_background.cor')

# define FastText Model
model = FastText(vector_size=100)
# using lee corpus to build the vocabulary
model.build_vocab(corpus_file=corpus_file)
# train the model
model.train(
            corpus_file=corpus_file,
            epochs=model.epochs,
            total_examples=model.corpus_count,
            total_words=model.corpus_total_words,
)

print(model)

FastText(vocab=1762, vector_size=100, alpha=0.025)


### 训练用超参数

训练用的超参数与Word2Vec类似：
- model：training architecture（cbow, skipgram)
- vector_size: 所要学习表征向量的维度（默认100）
- alpha: 初始学习率（默认0.025)
- window: 上下文窗口大小
- min_count: 忽略的最小出现次数
- loss：训练目标，ns, hs, softmax
- sample: downsampling 高频词汇的门限值
- negative：negative words取样的数量
- epochs：训练周期
- sorted_vocab: 已排序的词汇表
- threads：线程数
- min_n: 最小字符数量
- max_n: 最大字符数量
- bucket：用于hashing ngrams的bucket数量

### 词向量查找

所有用于fastText单词查找所必要的信息都存储在model.wv属性中，如果不需要继续训练模型，可以导出该属性：

In [2]:
wv = model.wv
print(wv)
print('night' in wv.key_to_index)
print(wv['night'])

True
[-0.2053806   0.18894379 -0.2690745  -0.09036077  0.06647884  0.376969
  0.2978653   0.49579558  0.2507173  -0.23425335  0.02566904 -0.16051692
 -0.228859    0.5148784  -0.3989852  -0.5597922   0.18645956 -0.2464775
 -0.42931154 -0.540943   -0.47003725 -0.05841109 -0.45448637 -0.12823395
 -0.20032988 -0.32459506 -0.6892999  -0.11594225 -0.3285938   0.27487138
 -0.3284593   0.30654034  0.8397898  -0.2658581   0.1870984   0.39889276
  0.38876215 -0.10158689 -0.37743023 -0.344173    0.4690904  -0.42798534
  0.03297168 -0.41184503 -0.5221864  -0.3052265  -0.078646    0.12564796
  0.3729068  -0.00506498  0.35743275 -0.43595463  0.29589665 -0.41202697
 -0.19048856 -0.18640569 -0.16040258 -0.13255174  0.04511935 -0.35875258
 -0.34318477 -0.44387108 -0.18265612  0.34746906 -0.12165342  0.6920488
  0.06088303  0.06516559  0.42835665  0.24624774 -0.23957959  0.38917154
  0.5097583  -0.6535613   0.34569028 -0.10344681  0.28131518 -0.04362335
  0.05995783  0.38687566  0.18292588 -0.49996376 -

###  相似度相关操作

相似度相关的操作与Word2Vec操作类似，并且OOV单词也可以用于相似度相关的操作(provided they have at least one character ngram present in the training data)：

print("nights" in wv.key_to_index)
print("night"  in wv.key_to_index)

尽管nights是OOV，但是night在训练集中，因此可以对nights进行相似度计算：

In [3]:
print(wv.similarity("night", "nights"))

0.99999195


### 其他的相似度操作

用于训练的语料库很小，因此结果仅用于提供proof-of-concept:

In [4]:
print(wv.most_similar("nights"))

[('night', 0.9999919533729553), ('rights', 0.9999879598617554), ('flights', 0.9999877214431763), ('overnight', 0.9999871850013733), ('fighting', 0.9999857544898987), ('fighters', 0.9999856352806091), ('entered', 0.9999853372573853), ('fight', 0.9999852180480957), ('fighter', 0.9999852180480957), ('eight', 0.9999844431877136)]


In [5]:
print(wv.n_similarity(['sushi', 'shop'], ['japanese', 'restarurant']))

0.9999374


In [6]:
print(wv.most_similar(positive=['baghdad', 'england'], negative=['london']))

[('capital,', 0.9996396899223328), ('find', 0.9996392726898193), ('findings', 0.9996320009231567), ('field', 0.999630331993103), ('seekers.', 0.999629557132721), ('finding', 0.9996291399002075), ('abuse', 0.9996284246444702), ('had', 0.9996267557144165), ('storm', 0.9996263980865479), ('heading', 0.9996235370635986)]


### 词移距离

计算两个句子的词移距离：

In [7]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

In [8]:
# 去除停用词
from gensim.parsing.preprocessing import STOPWORDS
sentence_obama = [w for w in sentence_obama if w not in STOPWORDS]
sentence_president = [ w for w in sentence_president if w not in STOPWORDS]

# 计算两个句子的词移距离
distance = wv.wmdistance(sentence_obama, sentence_president)
print(f"Word Movers Distence is {distance} (lower means closer)")

Word Movers Distence is 0.01599428586470615 (lower means closer)
